In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [16]:
data=pd.read_csv('accre-gpu-jobs-2022-v2.csv', on_bad_lines='skip')
data

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66M,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
1,32880701,glasshouse,brady,0,05:00:00,00:00:22,1,3,1,maxwell,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32M,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62M,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09M,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336945,46754520_26,harovin,porsha,0,5-00:00:00,00:07:07,1,6,1,turing,0:0,RUNNING
336946,46754542,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336947,46754560,malanga,arline,0,2-00:00:00,00:00:00,0,2,1,pascal,0:0,PENDING
336948,46754598,glasshouse,treva,145.24M,1-00:00:00,00:01:26,1,5,1,turing,0:0,COMPLETED


In [28]:
data = (data[
    (data['STATE']=='COMPLETED') &
    (data['USEDMEM']!='0') &
    (data['USEDMEM']!='0.00M')])

In [29]:
data

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66M,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32M,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62M,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09M,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED
5,32880721,glasshouse,brady,403.40M,3-12:00:00,18:49:36,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336933,46753813,harovin,shannon,137.96M,5-00:00:00,00:00:52,1,1,1,turing,0:0,COMPLETED
336934,46753820_26,harovin,porsha,5093.70M,5-00:00:00,00:34:51,1,6,1,turing,0:0,COMPLETED
336936,46753823,harovin,shannon,156.21M,5-00:00:00,00:04:03,1,1,1,turing,0:0,COMPLETED
336942,46754095,harovin,shannon,2820.52M,5-00:00:00,00:11:13,1,1,1,turing,0:0,COMPLETED


In [34]:
def str2seconds(timestr):
    if len(timestr) == 8:
        # split in hh, mm, ss
        hh, mm, ss = timestr.split(':')
        return int(hh) * 3600 + int(mm) * 60 + int(ss)
    else:
        d = timestr[0]
        dropday = timestr[2:]
        hh, mm, ss = dropday.split(':')
        return int(d) *3600*24 + int(hh) * 3600 + int(mm) * 60 + int(ss)

In [47]:
usedtime = data['USEDTIME']
usedtime_seconds = []
for string in usedtime:
    usedtime_seconds.append(str2seconds(string))
usedtime_seconds

[793,
 537,
 18586,
 15070,
 67776,
 29358,
 333,
 328,
 293,
 2441,
 4136,
 1495,
 1426,
 332,
 327,
 293,
 2473,
 4141,
 7205,
 334,
 326,
 293,
 2479,
 4121,
 332,
 326,
 293,
 2459,
 4129,
 21605,
 21605,
 21617,
 346,
 326,
 300,
 2453,
 3810,
 760,
 1520,
 746,
 360,
 325,
 299,
 2563,
 3475,
 777,
 758,
 3292,
 7204,
 741,
 718,
 2135,
 767,
 359,
 327,
 299,
 2580,
 3481,
 741,
 730,
 794,
 792,
 1405,
 81,
 776,
 762,
 362,
 350,
 296,
 2454,
 3489,
 731,
 734,
 252,
 732,
 21605,
 117514,
 7206,
 758,
 6240,
 754,
 778,
 722,
 50,
 14405,
 773,
 748,
 746,
 752,
 335,
 331,
 298,
 2434,
 3744,
 770,
 777,
 753,
 787,
 796,
 41,
 39,
 69,
 778,
 14419,
 750,
 786,
 742,
 635,
 537,
 19065,
 528,
 18275,
 778,
 338,
 331,
 296,
 2494,
 3635,
 2086,
 762,
 165,
 35410,
 45,
 98,
 761,
 107347,
 449,
 18521,
 445,
 18927,
 447,
 19260,
 294,
 749,
 7213,
 756,
 351,
 347,
 307,
 2498,
 3729,
 770,
 39,
 66,
 72,
 125,
 94,
 69,
 789,
 4668,
 82,
 787,
 784,
 807,
 396,
 394,
 313

In [41]:
reqtime = data['REQTIME']
reqtime_seconds = []
for string in reqtime:
    reqtime_seconds.append(str2seconds(string))
reqtime_seconds

[172800,
 18000,
 72000,
 54000,
 302400,
 172800,
 345600,
 345600,
 345600,
 345600,
 345600,
 172800,
 172800,
 345600,
 345600,
 345600,
 345600,
 345600,
 7200,
 345600,
 345600,
 345600,
 345600,
 345600,
 345600,
 345600,
 345600,
 345600,
 345600,
 21600,
 21600,
 21600,
 345600,
 345600,
 345600,
 345600,
 345600,
 172800,
 172800,
 172800,
 345600,
 345600,
 345600,
 345600,
 345600,
 172800,
 172800,
 3600,
 7200,
 172800,
 172800,
 172800,
 172800,
 345600,
 345600,
 345600,
 345600,
 345600,
 172800,
 172800,
 172800,
 172800,
 172800,
 172800,
 172800,
 172800,
 345600,
 345600,
 345600,
 345600,
 345600,
 172800,
 172800,
 3600,
 172800,
 21600,
 432000,
 7200,
 172800,
 7200,
 172800,
 172800,
 172800,
 172800,
 14400,
 172800,
 172800,
 172800,
 172800,
 345600,
 345600,
 345600,
 345600,
 345600,
 172800,
 172800,
 172800,
 172800,
 172800,
 172800,
 172800,
 172800,
 172800,
 14400,
 172800,
 172800,
 172800,
 10800,
 18000,
 72000,
 18000,
 72000,
 172800,
 345600,


In [46]:
data['USEDMEM'] = data['USEDMEM'].str.replace("M", '').astype('float')
data

/var/folders/y2/1hbr5nhn5g74yh6sf24m5s9xr861g0/T/ipykernel_69627/2559702012.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['USEDMEM'] = data['USEDMEM'].str.replace("M", '').astype('float')


,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE
0,32880657,malanga,arline,18.66,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED
2,32880718,glasshouse,brady,402.32,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED
3,32880719,glasshouse,brady,417.62,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED
4,32880720,glasshouse,brady,450.09,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED
5,32880721,glasshouse,brady,403.40,3-12:00:00,18:49:36,1,1,1,pascal,0:0,COMPLETED
...,...,...,...,...,...,...,...,...,...,...,...,...
336933,46753813,harovin,shannon,137.96,5-00:00:00,00:00:52,1,1,1,turing,0:0,COMPLETED
336934,46753820_26,harovin,porsha,5093.70,5-00:00:00,00:34:51,1,6,1,turing,0:0,COMPLETED
336936,46753823,harovin,shannon,156.21,5-00:00:00,00:04:03,1,1,1,turing,0:0,COMPLETED
336942,46754095,harovin,shannon,2820.52,5-00:00:00,00:11:13,1,1,1,turing,0:0,COMPLETED


In [48]:
data['used_time']=usedtime_seconds
data

/var/folders/y2/1hbr5nhn5g74yh6sf24m5s9xr861g0/T/ipykernel_69627/4122223406.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['used_time']=usedtime_seconds


,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE,used_time
0,32880657,malanga,arline,18.66,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED,793
2,32880718,glasshouse,brady,402.32,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED,537
3,32880719,glasshouse,brady,417.62,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED,18586
4,32880720,glasshouse,brady,450.09,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED,15070
5,32880721,glasshouse,brady,403.40,3-12:00:00,18:49:36,1,1,1,pascal,0:0,COMPLETED,67776
...,...,...,...,...,...,...,...,...,...,...,...,...,...
336933,46753813,harovin,shannon,137.96,5-00:00:00,00:00:52,1,1,1,turing,0:0,COMPLETED,52
336934,46753820_26,harovin,porsha,5093.70,5-00:00:00,00:34:51,1,6,1,turing,0:0,COMPLETED,2091
336936,46753823,harovin,shannon,156.21,5-00:00:00,00:04:03,1,1,1,turing,0:0,COMPLETED,243
336942,46754095,harovin,shannon,2820.52,5-00:00:00,00:11:13,1,1,1,turing,0:0,COMPLETED,673


In [52]:
data['weighted']= (data['used_time'] * data['USEDMEM']) / data['used_time'].sum()
data

/var/folders/y2/1hbr5nhn5g74yh6sf24m5s9xr861g0/T/ipykernel_69627/206135541.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['weighted']= (data['used_time'] * data['USEDMEM']) / data['used_time'].sum()


,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE,used_time,weighted
0,32880657,malanga,arline,18.66,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED,793,0.000009
2,32880718,glasshouse,brady,402.32,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED,537,0.000135
3,32880719,glasshouse,brady,417.62,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED,18586,0.004867
4,32880720,glasshouse,brady,450.09,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED,15070,0.004253
5,32880721,glasshouse,brady,403.40,3-12:00:00,18:49:36,1,1,1,pascal,0:0,COMPLETED,67776,0.017143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336933,46753813,harovin,shannon,137.96,5-00:00:00,00:00:52,1,1,1,turing,0:0,COMPLETED,52,0.000004
336934,46753820_26,harovin,porsha,5093.70,5-00:00:00,00:34:51,1,6,1,turing,0:0,COMPLETED,2091,0.006678
336936,46753823,harovin,shannon,156.21,5-00:00:00,00:04:03,1,1,1,turing,0:0,COMPLETED,243,0.000024
336942,46754095,harovin,shannon,2820.52,5-00:00:00,00:11:13,1,1,1,turing,0:0,COMPLETED,673,0.001190


In [53]:
data['per_gpu'] = data['weighted'] / data['GPUS']
data

/var/folders/y2/1hbr5nhn5g74yh6sf24m5s9xr861g0/T/ipykernel_69627/802593730.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['per_gpu'] = data['weighted'] / data['GPUS']


,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE,used_time,weighted,per_gpu
0,32880657,malanga,arline,18.66,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED,793,0.000009,0.000009
2,32880718,glasshouse,brady,402.32,05:00:00,00:08:57,1,3,1,maxwell,0:0,COMPLETED,537,0.000135,0.000135
3,32880719,glasshouse,brady,417.62,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED,18586,0.004867,0.004867
4,32880720,glasshouse,brady,450.09,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED,15070,0.004253,0.004253
5,32880721,glasshouse,brady,403.40,3-12:00:00,18:49:36,1,1,1,pascal,0:0,COMPLETED,67776,0.017143,0.017143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336933,46753813,harovin,shannon,137.96,5-00:00:00,00:00:52,1,1,1,turing,0:0,COMPLETED,52,0.000004,0.000004
336934,46753820_26,harovin,porsha,5093.70,5-00:00:00,00:34:51,1,6,1,turing,0:0,COMPLETED,2091,0.006678,0.006678
336936,46753823,harovin,shannon,156.21,5-00:00:00,00:04:03,1,1,1,turing,0:0,COMPLETED,243,0.000024,0.000024
336942,46754095,harovin,shannon,2820.52,5-00:00:00,00:11:13,1,1,1,turing,0:0,COMPLETED,673,0.001190,0.001190


In [54]:
pascal=data[data['PARTITION']=='pascal']
pascal

,JOBID,ACCOUNT,USER,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,GPUS,PARTITION,EXITCODE,STATE,used_time,weighted,per_gpu
0,32880657,malanga,arline,18.66,2-00:00:00,00:13:13,1,2,1,pascal,0:0,COMPLETED,793,9.278040e-06,9.278040e-06
3,32880719,glasshouse,brady,417.62,20:00:00,05:09:46,1,1,1,pascal,0:0,COMPLETED,18586,4.866745e-03,4.866745e-03
4,32880720,glasshouse,brady,450.09,15:00:00,04:11:10,1,1,1,pascal,0:0,COMPLETED,15070,4.252889e-03,4.252889e-03
5,32880721,glasshouse,brady,403.40,3-12:00:00,18:49:36,1,1,1,pascal,0:0,COMPLETED,67776,1.714286e-02,1.714286e-02
7,32880741,malanga,arline,1119.66,2-00:00:00,08:09:18,1,2,1,pascal,0:0,COMPLETED,29358,2.061029e-02,2.061029e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336914,46753151,malanga,arline,29.85,2-00:00:00,00:14:28,1,2,1,pascal,0:0,COMPLETED,868,1.624559e-05,1.624559e-05
336915,46753169,malanga,arline,28.11,2-00:00:00,00:14:16,1,2,1,pascal,0:0,COMPLETED,856,1.508711e-05,1.508711e-05
336916,46753170,glasshouse,tamara,10007.76,20:00:00,00:29:16,1,5,4,pascal,0:0,COMPLETED,1756,1.101876e-02,2.754690e-03
336917,46753171,malanga,arline,28.21,2-00:00:00,00:14:29,1,2,1,pascal,0:0,COMPLETED,869,1.537072e-05,1.537072e-05
